In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [15]:
np.random.seed(42)
tf.random.set_seed(42)

In [16]:
# Define your RegNet architecture
input_layer = Input(shape=(224, 224, 3))

In [17]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, MaxPooling2D, GlobalAveragePooling2D, Dense


In [18]:
# Define the input layer
input_layer = Input(shape=(224, 224, 3))


In [19]:
# Initial stem
x = Conv2D(filters=32, kernel_size=(3, 3), strides=(2, 2), padding='same')(input_layer)
x = BatchNormalization()(x)
x = ReLU()(x)

In [20]:
# Stages
num_stages = 4  # You can adjust the number of stages
num_blocks_per_stage = 3  # You can adjust the number of blocks per stage
width_per_group = 16  # You can adjust the width per group


In [22]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, Add

def ResidualBlock(input_layer, stride, out_channels):
    # Shortcut connection (identity)
    shortcut = input_layer
    
    # First convolutional layer
    x = Conv2D(out_channels, kernel_size=(3, 3), strides=stride, padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Second convolutional layer
    x = Conv2D(out_channels, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    
    # Residual connection
    if stride != (1, 1) or input_layer.shape[-1] != out_channels:
        shortcut = Conv2D(out_channels, kernel_size=(1, 1), strides=stride, padding='same')(input_layer)
    
    # Add the shortcut to the output
    x = Add()([x, shortcut])
    x = ReLU()(x)
    
    return x

In [23]:
for s in range(num_stages):
    for b in range(num_blocks_per_stage):
        stride = (2, 2) if s > 0 and b == 0 else (1, 1)
        x = ResidualBlock(x, stride, width_per_group * 2 ** s)


In [24]:
# Global Average Pooling
x = GlobalAveragePooling2D()(x)

# Fully connected layer for classification
output_layer = Dense(2, activation='softmax')(x)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

In [25]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [26]:
# Data preprocessing and loading (similar to your CNN model)
IMAGE_SIZE = (224, 224)
train_path = 'chest_xray/train'
valid_path = 'chest_xray/test'

In [27]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(train_path, target_size=IMAGE_SIZE, batch_size=10, class_mode='categorical')
test_set = test_datagen.flow_from_directory(valid_path, target_size=IMAGE_SIZE, batch_size=10, class_mode='categorical')


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [28]:
# Training the model
model.fit(
    training_set,
    validation_data=test_set,
    epochs=1,  # You can adjust the number of epochs
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

522/522 [==============================] - 200s 379ms/step - loss: 0.3249 - accuracy: 0.8593 - val_loss: 7.6394 - val_accuracy: 0.3814
